In [1]:
import time
start_time = time.time()

# Preparation - Data grouped on forum level

Import the packages and load the data

In [2]:
# Import needed READkages
import os
import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.preprocessing import KBinsDiscretizer
import seaborn as sns; sns.set()
import glob
from sklearn.linear_model import LinearRegression
from statsmodels.formula.api import ols
import re
import nltk
from langdetect import detect
from sklearn import feature_extraction, model_selection, naive_bayes, pipeline, manifold, preprocessing
from lime import lime_text
import gensim
import gensim.downloader as gensim_api
from tensorflow.keras import models, layers, preprocessing as kprocessing
from tensorflow.keras import backend as K
import transformers
plt.style.use('seaborn-whitegrid')
sns.set_style("whitegrid")
from bs4 import BeautifulSoup
from tqdm import tqdm
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem.snowball import EnglishStemmer
import spacy
from spacy import displacy
from wordcloud import WordCloud
from sklearn.preprocessing import KBinsDiscretizer
from nbpep8.nbpep8 import pep8
from sklearn.metrics.pairwise import cosine_similarity
from linearmodels import PanelOLS
from linearmodels.panel import PanelOLS
from linearmodels.panel import PooledOLS
import statsmodels.api as sm
from statsmodels.tools.tools import add_constant
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.stats.diagnostic import het_breuschpagan

In [3]:
#Set the working directory
os.chdir('C:\\Users\\piotr\\OneDrive - Erasmus University Rotterdam\\Thesis\\')

Load the grouped data frames

In [4]:
#Load the estimation data frames
est_grouped_df_week_tag_reduced_NEW = pd.read_csv('Master Thesis Data\\grouped_df_tag_reduced_week_NEW_2.csv')

In [5]:
est_grouped_df_week_tag_reduced_NEW.shape

(781821, 20)

Important modification - add weeknumber

In [6]:
from datetime import datetime

# Your function to modify a dataframe
def modify_dataframe(df):
      # Extract the start date of each week and convert it to datetime
    df['Week_Start_Date'] = df['Post_Creation_Date'].apply(lambda x: datetime.strptime(x.split('/')[0], '%Y-%m-%d'))

    # Find the minimum date in the DataFrame
    min_date = df['Week_Start_Date'].min()

    # Create a new column 'Week_Number' that calculates the difference in weeks from the minimum date
    df['Week_Number'] = (df['Week_Start_Date'] - min_date).dt.days // 7 + 1

    # Add the interaction term
    df['DiD'] = df['D'] * df['T']
    
    return df


# Create a copy of your dataframe
df = est_grouped_df_week_tag_reduced_NEW.copy()

# Modify the dataframe
est_grouped_df_week_tag_reduced_NEW = modify_dataframe(df)


In [7]:
def map_week_of_year_to_counter(week_of_year):
    if 27 <= week_of_year <= 52:
        return week_of_year - 26
    elif 1 <= week_of_year <= 16:
        return week_of_year + 26
    else:
        return None

est_grouped_df_week_tag_reduced_NEW['Week_Counter'] = est_grouped_df_week_tag_reduced_NEW['Week_of_Year'].apply(map_week_of_year_to_counter)

In [8]:
#Display all columns
pd.set_option('display.max_columns', None)
est_grouped_df_week_tag_reduced_NEW

,tag_combined,Post_Creation_Date,Post_Count,Post_Answer_Count,ari,fre,fk_grade,gunning_fog,D,T,smog_index,coleman_liau_index,Week_of_Year,Post_View_Count,Post_Score,Response_Scores,Post_Author,Answer_Author,Title_Length,Body_Length,Week_Start_Date,Week_Number,DiD,Week_Counter
0,.d.tstypescript,2021-07-26/2021-08-01,1,2,6.1,78.96,4.6,4.34,0.0,0.0,6.2,7.16,30,161,0,0,1,1,22.0,1152.0,2021-07-26,4,0.0,4
1,.d.tstypescript,2021-08-09/2021-08-15,1,2,12.2,65.46,9.7,12.23,0.0,0.0,11.2,9.75,32,7671,11,14,1,1,34.0,850.0,2021-08-09,6,0.0,6
2,.d.tstypescript,2021-08-23/2021-08-29,1,1,11.9,62.92,10.7,11.40,0.0,0.0,10.3,7.55,34,899,1,0,1,1,71.0,2336.0,2021-08-23,8,0.0,8
3,.d.tstypescript,2022-01-24/2022-01-30,1,0,9.8,63.70,8.4,11.40,0.0,1.0,12.5,10.09,4,42,1,0,1,1,30.0,839.0,2022-01-24,30,0.0,30
4,.d.tstypescript,2022-02-07/2022-02-13,1,0,3.9,90.46,2.2,2.52,0.0,1.0,6.0,5.45,6,1090,2,0,1,1,48.0,438.0,2022-02-07,32,0.0,32
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
781816,сеть,2022-01-31/2022-02-06,1,1,0.0,206.84,-15.7,0.00,0.0,1.0,0.0,-15.81,5,29,0,1,1,1,55.0,562.0,2022-01-31,31,0.0,31
781817,сеть,2022-03-21/2022-03-27,1,0,0.0,206.84,-15.7,0.00,0.0,1.0,0.0,-15.81,12,17,0,0,1,1,38.0,551.0,2022-03-21,38,0.0,38
781818,сеть,2022-08-08/2022-08-14,1,1,0.0,206.84,-15.7,0.00,1.0,0.0,0.0,-15.81,32,30,2,4,1,1,83.0,193.0,2022-08-08,58,0.0,6
781819,сеть,2022-08-29/2022-09-04,1,0,0.0,206.84,-15.7,0.00,1.0,0.0,0.0,-15.81,35,27,0,0,1,1,26.0,2076.0,2022-08-29,61,0.0,9


## Average the readability metrics

Columns with different redability metrics:

ari	

fre	 (1-fre)

fk_grade

gunning_fog

smog_index	

coleman_liau_index

In [9]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

# Instantiate the scaler
scaler = MinMaxScaler()

# List of columns to normalize
cols_to_normalize = ['ari', 'fre', 'fk_grade', 'gunning_fog', 'smog_index', 'coleman_liau_index']

# Apply the scaler to the DataFrame
est_grouped_df_week_tag_reduced_NEW[cols_to_normalize] = scaler.fit_transform(est_grouped_df_week_tag_reduced_NEW[cols_to_normalize])

# Invert the 'fre' column
est_grouped_df_week_tag_reduced_NEW['fre'] = 1 - est_grouped_df_week_tag_reduced_NEW['fre']

# Add a new column with the average of the normalized columns
est_grouped_df_week_tag_reduced_NEW['avg_readability'] = est_grouped_df_week_tag_reduced_NEW[cols_to_normalize].mean(axis=1)

# Choose the data frame

Note that the data ranges from 01-07-2021 to 30-04-2022 and from 01-07-2022 to 30-04-2023. That implies that the weeks 18,19,20,21,22,23,24,25 are not present in the data set.
Weeks 17 and 26 are dropped as they are incomplete

# 8.1 PTA - tag reduced level

In [16]:
df = est_grouped_df_week_tag_reduced_NEW.copy()

df['DiD*Week_Counter'] = df['DiD'] * df['Week_Counter']

#Set index to tag_combined and Week_of_Year
df = df.set_index(['tag_combined', 'Week_of_Year'])


In [17]:
df

Post_Creation_Date  Post_Count  \
tag_combined    Week_of_Year                                      
.d.tstypescript 30            2021-07-26/2021-08-01           1   
                32            2021-08-09/2021-08-15           1   
                34            2021-08-23/2021-08-29           1   
                4             2022-01-24/2022-01-30           1   
                6             2022-02-07/2022-02-13           1   
...                                             ...         ...   
сеть            5             2022-01-31/2022-02-06           1   
                12            2022-03-21/2022-03-27           1   
                32            2022-08-08/2022-08-14           1   
                35            2022-08-29/2022-09-04           1   
                5             2023-01-30/2023-02-05           2   

                              Post_Answer_Count       ari           fre  \
tag_combined    Week_of_Year                                              
.d.tstypescript 30                            2  0.010524  4.960595e-02   
                32                            2  0.014151  5.484273e-02   
                34                            1  0.013972  5.582803e-02   
                4                             0  0.012724  5.552546e-02   
                6                             0  0.009216  4.514498e-02   
...                                         ...       ...           ...   
сеть            5                             1  0.006897  1.110223e-16   
                12                            0  0.006897  1.110223e-16   
                32                            1  0.006897  1.110223e-16   
                35                            0  0.006897  1.110223e-16   
                5                             0  0.006897  1.110223e-16   

                              fk_grade  gunning_fog    D    T  smog_index  \
tag_combined    Week_of_Year                                                
.d.tstypescript 30            0.055995     0.016630  0.0  0.0    0.143852   
                32            0.070063     0.046863  0.0  0.0    0.259861   
                34            0.072821     0.043683  0.0  0.0    0.238979   
                4             0.066477     0.043683  0.0  1.0    0.290023   
                6             0.049375     0.009656  0.0  1.0    0.139211   
...                                ...          ...  ...  ...         ...   
сеть            5             0.000000     0.000000  0.0  1.0    0.000000   
                12            0.000000     0.000000  0.0  1.0    0.000000   
                32            0.000000     0.000000  1.0  0.0    0.000000   
                35            0.000000     0.000000  1.0  0.0    0.000000   
                5             0.000000     0.000000  1.0  1.0    0.000000   

                              coleman_liau_index  Post_View_Count  Post_Score  \
tag_combined    Week_of_Year                                                    
.d.tstypescript 30                      0.019714              161           0   
                32                      0.020960             7671          11   
                34                      0.019902              899           1   
                4                       0.021124               42           1   
                6                       0.018891             1090           2   
...                                          ...              ...         ...   
сеть            5                       0.008661               29           0   
                12                      0.008661               17           0   
                32                      0.008661               30           2   
                35                      0.008661               27           0   
                5                       0.008661               31           0   

                              Response_Scores  Post_Author  Answer_Author  \
tag_combined    Week_of_Year                        

## 8.1.1 Post Count

In [18]:
# Define your dependent variables
dependent_vars = ['Post_Count', 'Post_View_Count', 'Post_Score', 'Post_Answer_Count', 'Response_Scores', 'avg_readability']

# Empty dictionary to store results
summary_dfs = {}

# Loop through the dependent variables
for dv in dependent_vars:

    # Define dependent and independent variables
    Y = df[dv]
    X = df[['D', 'T', 'DiD*Week_Counter']]

    # Run the model
    model = PanelOLS(Y, X, entity_effects=True, time_effects=True)
    result = model.fit()

    # Creating the summary DataFrame
    summary_df = pd.DataFrame({'coefficients': result.params, 'p-values': result.pvalues})

    # Assigning the stars
    conditions = [
        (summary_df['p-values'] < 0.01),
        (summary_df['p-values'] < 0.05),
        (summary_df['p-values'] < 0.1)
    ]
    choices = ['***', '**', '*']
    summary_df['p-value sig.'] = np.select(conditions, choices, default='')

    # Round the coefficients to 4 decimal places
    summary_df['coefficients'] = summary_df['coefficients'].round(4)

    # Concatenating the stars to the coefficients
    summary_df['coefficients'] = summary_df['coefficients'].astype(str) + summary_df['p-value sig.']

    # Adding F-statistic, its p-value, and R^2
    f_statistic = round(result.f_statistic.stat, 4)
    f_pvalue = round(result.f_statistic.pval, 4)
    r_squared = round(result.rsquared, 4)

    # Assigning the stars to F-statistic
    f_stat_star = '***' if f_pvalue < 0.01 else '**' if f_pvalue < 0.05 else '*' if f_pvalue < 0.1 else ''

    additional_stats = pd.DataFrame({
        'coefficients': [f'{f_statistic}{f_stat_star}', r_squared],
        'p-values': [np.nan, np.nan],
        'p-value sig.': ['', '']
    }, index=['F-statistic', 'R-squared'])

    summary_df = summary_df.append(additional_stats)

    # Add the name of dependent variable as the first row
    dep_var_row = pd.DataFrame({
        'coefficients': [result.model.dependent.vars[0]],
        'p-values': [np.nan],
        'p-value sig.': ['']
    }, index=['Dependent Variable'])

    summary_df = pd.concat([dep_var_row, summary_df])

    # Drop the unnecessary columns
    summary_df = summary_df.drop(columns=['p-values', 'p-value sig.'])

    # Save to dictionary
    summary_dfs[dv] = summary_df


C:\Users\piotr\AppData\Local\Temp\ipykernel_3876\1201411805.py:50: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  summary_df = summary_df.append(additional_stats)
C:\Users\piotr\AppData\Local\Temp\ipykernel_3876\1201411805.py:50: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  summary_df = summary_df.append(additional_stats)
C:\Users\piotr\AppData\Local\Temp\ipykernel_3876\1201411805.py:50: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  summary_df = summary_df.append(additional_stats)
C:\Users\piotr\AppData\Local\Temp\ipykernel_3876\1201411805.py:50: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  summary_df = summary_df.append(additional_sta

In [19]:
# Concatenate all summary dataframes into one
summary_linear_trend = pd.concat(summary_dfs)

# Save the concatenated dataframe to an Excel file
with pd.ExcelWriter('summary_linear_trend.xlsx') as writer:
    summary_linear_trend.to_excel(writer)
